In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load data
data = pd.read_csv('prediction_dataset.csv')

# Create an ID column
data['id'] = range(1, len(data) + 1)

# Move 'id' column to the first position
cols = ['id'] + [col for col in data.columns if col != 'id']
data = data[cols]

# One-hot encode categorical features
categorical_cols = ['day_of_week', 'is_weekend', 'month', 'year', 'season']
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Ensure categorical columns are 0 and 1, not True and False
for col in data.select_dtypes(include=['bool']).columns:
    data[col] = data[col].astype(int)

# Convert datetime_utc to datetime64
data['datetime_utc'] = pd.to_datetime(data['datetime_utc'], utc=True).dt.tz_localize(None)

# Select numerical columns to standardize (excluding target 'spot_price' and 'id')
numerical_columns = [
    'volume_demand', 'volume_production', 'price_lag_1d', 'price_lag_7d', 'price_lag_30d',
    'price_rolling_mean_7d', 'price_rolling_std_7d', 'price_rolling_mean_30d', 'price_rolling_std_30d',
    'natural_gas_price', 'Fill_Level', 'Precipitation', 'Max_Temp', 'Avg_Temp', 'Min_Temp',
    'Max_Wind', 'Max_Gust', 'Avg_Wind', 'Snow_Depth'
]

# Initialize the scaler
scaler = StandardScaler()

# Apply standardization only to the numerical features
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Handle missing values if any
imputer = SimpleImputer(strategy='mean')
data[numerical_columns] = imputer.fit_transform(data[numerical_columns])

# Print the first few rows and data info to verify the changes
print(data.head())
print(data.info())

# Save the preprocessed dataset with the spot price as a CSV
data.to_csv('preprocessed_dataset.csv', index=False)
print("Preprocessed dataset saved as 'preprocessed_dataset.csv'")


   id datetime_utc  volume_demand  volume_production  spot_price  \
0   1   2016-01-30       0.842783          -0.373537   15.345417   
1   2   2016-01-31       0.829238          -0.051549   16.520833   
2   3   2016-02-01       1.381032           0.839492   18.585417   
3   4   2016-02-02       1.196450           0.382193   16.956250   
4   5   2016-02-03       1.296145           0.846770   17.781250   

   price_lag_1d  price_lag_7d  price_lag_30d  price_rolling_mean_7d  \
0     -1.644997     -0.783081      -1.663573              -1.427364   
1     -1.737056     -0.969281      -1.635576              -1.520384   
2     -1.606686     -0.759683      -1.785496              -1.610033   
3     -1.377694     -1.430905      -1.705117              -1.628853   
4     -1.558392     -1.589384      -1.392131              -1.611132   

   price_rolling_std_7d  ...  month_8  month_9  month_10  month_11  month_12  \
0              1.066517  ...        0        0         0         0         0   
1   

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load data
data = pd.read_csv("prediction_dataset.csv")

# Create an ID column
data["item_id"] = range(1, len(data) + 1)

# Rename columns to match AutoGluon TimeSeriesDataFrame format
data = data.rename(columns={"datetime_utc": "timestamp", "spot_price": "target"})

# Move 'item_id' and 'timestamp' columns to the first positions
cols = ["item_id", "timestamp"] + [
    col for col in data.columns if col not in ["item_id", "timestamp"]
]
data = data[cols]

# Convert timestamp to datetime
data["timestamp"] = pd.to_datetime(data["timestamp"], utc=True).dt.tz_localize(None)

# Define categorical columns
categorical_cols = ['day_of_week', 'is_weekend', 'month', 'year', 'season']

# Check which categorical columns are actually present in the data
available_categorical_cols = [col for col in categorical_cols if col in data.columns]

# One-hot encode available categorical features
if available_categorical_cols:
    data = pd.get_dummies(data, columns=available_categorical_cols, drop_first=True)

# Ensure categorical columns are 0 and 1, not True and False
for col in data.select_dtypes(include=["bool"]).columns:
    data[col] = data[col].astype(int)

# Select numerical columns to standardize (excluding target 'target' and 'item_id')
# numerical_columns = [
#     'volume_demand', 'volume_production', 'price_lag_1d', 'price_lag_7d', 'price_lag_30d',
#     'price_rolling_mean_7d', 'price_rolling_std_7d', 'price_rolling_mean_30d', 'price_rolling_std_30d',
#     'natural_gas_price', 'Fill_Level', 'Precipitation', 'Max_Temp', 'Avg_Temp', 'Min_Temp',
#     'Max_Wind', 'Max_Gust', 'Avg_Wind', 'Snow_Depth'
# ]

# Keep only the columns we want to standardize
numerical_columns = [
    "volume_demand",
    "volume_production",
    "price_lag_1d",
    "price_lag_7d",
    "price_lag_30d",
    "Fill_Level",
    "Avg_Temp",
]

# Update selected_columns to use only available columns
selected_columns = ['item_id', 'timestamp', 'target'] + [col for col in data.columns if col in numerical_columns or col.startswith(tuple(available_categorical_cols))]
data = data[selected_columns]

# Initialize the scaler
scaler = StandardScaler()

# Apply standardization only to the numerical features
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Handle missing values if any
imputer = SimpleImputer(strategy="mean")
data[numerical_columns] = imputer.fit_transform(data[numerical_columns])

# Print the first few rows and data info to verify the changes
print(data.head())
print(data.info())

# Save the preprocessed dataset as a CSV
data.to_csv("automl_dataset.csv", index=False)
print("Preprocessed dataset saved as 'automl_dataset.csv'")

   item_id  timestamp     target  volume_demand  volume_production  \
0        1 2016-01-30  15.345417       0.842783          -0.373537   
1        2 2016-01-31  16.520833       0.829238          -0.051549   
2        3 2016-02-01  18.585417       1.381032           0.839492   
3        4 2016-02-02  16.956250       1.196450           0.382193   
4        5 2016-02-03  17.781250       1.296145           0.846770   

   price_lag_1d  price_lag_7d  price_lag_30d  month_sin  month_cos  ...  \
0     -1.644997     -0.783081      -1.663573        0.0   1.000000  ...   
1     -1.737056     -0.969281      -1.635576        0.0   1.000000  ...   
2     -1.606686     -0.759683      -1.785496        0.5   0.866025  ...   
3     -1.377694     -1.430905      -1.705117        0.5   0.866025  ...   
4     -1.558392     -1.589384      -1.392131        0.5   0.866025  ...   

   month_8  month_9  month_10  month_11  month_12  year_2017  year_2018  \
0        0        0         0         0         0    